# 28-仿射变换（ Afine Transformations ）——平行移动

利用仿射变换让图像在$x$方向上$+30$，在$y$方向上$-30$吧！

仿射变换利用$3\times3$的矩阵来进行图像变换。

变换的方式有平行移动（问题二十八）、放大缩小（问题二十九）、旋转（问题三十）、倾斜（问题三十一）等。

原图像记为$(x,y)$，变换后的图像记为$(x',y')$。

图像放大缩小矩阵为下式： $$ \left( \begin{matrix} x'\ y' \end{matrix} \right)= \left( \begin{matrix} a&b\ c&d \end{matrix} \right)\ \left( \begin{matrix} x\ y \end{matrix} \right) $$ 另一方面，平行移动按照下面的式子计算： $$ \left( \begin{matrix} x'\ y' \end{matrix} \right)= \left( \begin{matrix} x\ y \end{matrix} \right)+ \left( \begin{matrix} t_x\ t_y \end{matrix} \right) $$ 把上面两个式子盘成一个： $$ \left( \begin{matrix} x'\ y'\ 1 \end{matrix} \right)= \left( \begin{matrix} a&b&t_x\ c&d&t_y\ 0&0&1 \end{matrix} \right)\ \left( \begin{matrix} x\ y\ 1 \end{matrix} \right) $$ 但是在实际操作的过程中，如果一个一个地计算原图像的像素的话，处理后的像素可能没有在原图像中有对应的坐标。^2

因此，我们有必要对处理后的图像中各个像素进行仿射变换逆变换，取得变换后图像中的像素在原图像中的坐标。仿射变换的逆变换如下： $$ \left( \begin{matrix} x\ y \end{matrix} \right)= \frac{1}{a\ d-b\ c}\ \left( \begin{matrix} d&-b\ -c&a \end{matrix} \right)\
\left( \begin{matrix} x'\ y' \end{matrix} \right)- \left( \begin{matrix} t_x\ t_y \end{matrix} \right) $$ 这回的平行移动操作使用下面的式子计算。$t_x$和$t_y$是像素移动的距离。 $$ \left( \begin{matrix} x'\ y'\ 1 \end{matrix} \right)= \left( \begin{matrix} 1&0&t_x\ 0&1&t_y\ 0&0&1 \end{matrix} \right)\
\left( \begin{matrix} x\ y\ 1 \end{matrix} \right) $$

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [2]:
img = cv2.imread('../picture/chans.png').astype(np.float32)

In [3]:
def affine(img, a, b, c, d, tx, ty):
	H, W, C = img.shape

	# temporary image
	img_ = np.zeros((H+2, W+2, C), dtype=np.float32)
	img_[1:H+1, 1:W+1] = img

	# get new image shape
	H_new = np.round(H * d).astype(np.int)
	W_new = np.round(W * a).astype(np.int)
	out = np.zeros((H_new+1, W_new+1, C), dtype=np.float32)

	# get position of new image
	x_new = np.tile(np.arange(W_new), (H_new, 1))
	y_new = np.arange(H_new).repeat(W_new).reshape(H_new, -1)

	# get position of original image by affine
	adbc = a * d - b * c
	x = np.round((d * x_new  - b * y_new) / adbc).astype(np.int) - tx + 1
	y = np.round((-c * x_new + a * y_new) / adbc).astype(np.int) - ty + 1

	x = np.minimum(np.maximum(x, 0), W+1).astype(np.int)
	y = np.minimum(np.maximum(y, 0), H+1).astype(np.int)

	# assgin pixcel to new image
	out[y_new, x_new] = img_[y, x]

	out = out[:H_new, :W_new]
	out = out.astype(np.uint8)

	return out

In [10]:
img_affine = affine(img, a=1, b=0, c=0, d=1, tx=30, ty=-30)

In [11]:
cv2.imwrite('../picture/chan_result28_Afine Transformations.jpg', img_affine)
cv2.namedWindow("result", 0);
cv2.resizeWindow("result", (600, 600));
cv2.imshow("result", img_affine)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 29-仿射变换（ Afine Transformations ）——放大缩小

1. 使用仿射变换，将图片在$x$方向上放大$1.3$倍，在$y$方向上缩小至原来的$\frac{4}{5}$。
2. 在上面的条件下，同时在$x$方向上向右平移$30$（$+30$），在$y$方向上向上平移$30$（$-30$）。

In [4]:
img_af2 = affine(img, a=1.3, b=0, c=0, d=0.8, tx=30, ty=-30)

In [5]:
cv2.imwrite('../picture/chan_result29_Afine Transformations.jpg', img_af2)
cv2.namedWindow("result", 0);
cv2.resizeWindow("result", (600, 600));
cv2.imshow("result", img_af2)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 30-仿射变换（ Afine Transformations ）——旋转

1. 使用仿射变换，逆时针旋转$30$度。
2. 使用仿射变换，逆时针旋转$30$度并且能让全部图像显现（也就是说，单纯地做仿射变换会让图片边缘丢失，这一步中要让图像的边缘不丢失，需要耗费一些工夫）。
使用下面的式子进行逆时针方向旋转$A$度的仿射变换： $$ \left( \begin{matrix} x'\ y'\ 1 \end{matrix} \right)= \left( \begin{matrix} \cos(A)&-\sin(A)&t_x\ \sin(A)&\cos(A)&t_y\ 0&0&1 \end{matrix} \right)\ \left( \begin{matrix} x\ y\ 1 \end{matrix} \right) $$

In [6]:
A = 30.
theta = - np.pi * A / 180.

img_af3 = affine(img, a=np.cos(theta), b=-np.sin(theta), c=np.sin(theta), d=np.cos(theta), tx=0, ty=0)

In [7]:
cv2.imwrite('../picture/chan_result30_Afine Transformations.jpg', img_af3)
cv2.namedWindow("result", 0);
cv2.resizeWindow("result", (600, 600));
cv2.imshow("result", img_af3)
cv2.waitKey(0)
cv2.destroyAllWindows()